In [ ]:
%sh pip install --upgrade google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Not uninstalling protobuf at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2d0a7ebb-7e1e-460e-9387-68f1d7ae9578
    Can't uninstall 'protobuf'. No files were found to uninstall.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import io
import os

In [ ]:
from google.cloud import storage

In [ ]:
# Configura la variable de entorno GOOGLE_APPLICATION_CREDENTIALS con la ruta al archivo de credenciales JSON
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Workspace/Users/ccfontanillar@unadvirtual.edu.co/project-test-419903-c2e3452605db.json"

def get_filtered_urls(url):
    """
    Obtiene las URLs filtradas de la página web https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

    Args:
    url (str): URL de la página web.

    Returns:
    list: Lista de URLs filtradas.
    """
    filtered_urls = []

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', href=True)

        for link in links:
            href = link.get('href')
            absolute_url = urljoin(url, href)
            # Filtrar los enlaces por los que contienen "yellow_tripdata", "green_tripdata" o "fhvhv_tripdata" y el año 2024
            if any(keyword in absolute_url for keyword in ["yellow_tripdata", "green_tripdata", "fhvhv_tripdata"]) and "2024" in absolute_url:
                filtered_urls.append(absolute_url)

    return filtered_urls

def process_and_upload_trip_data(filtered_urls):
    """
    Lee cada URL en un DataFrame y lo convierte en formato Parquet antes de subirlo al Cloud Storage.

    Args:
    filtered_urls (list): Lista de URLs filtradas.

    Returns:
    None
    """
    try:
        # Crea un cliente de Cloud Storage
        storage_client = storage.Client()

        # Nombre de tu bucket en GCS
        bucket_name = 'upload-scraping-gcp'

        # Obtiene el bucket
        bucket = storage_client.get_bucket(bucket_name)

        # Recorre las URLs filtradas, lee cada una en un DataFrame y la guarda en un archivo Parquet en GCS
        for url in filtered_urls:
            try:
                # Leer el archivo Parquet en un DataFrame de Pandas
                df = pd.read_parquet(url)

                # Generar un nombre de archivo único para el Parquet
                parquet_file_name = os.path.basename(url)

                # Subir el archivo Parquet a GCS
                blob = bucket.blob(f"import/raw/{parquet_file_name}")
                blob.upload_from_string(df.to_parquet(), content_type='application/octet-stream')
                print(f"Archivo Parquet subido a Cloud Storage: gs://{bucket_name}/import/raw/{parquet_file_name}")

            except Exception as e:
                print(f"Error al procesar {url}: {e}")

    except Exception as e:
        print(f"Error en el proceso de carga a Cloud Storage: {e}")

# URL del sitio web que deseas scrape
url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

# Obtiene las URLs filtradas
filtered_urls = get_filtered_urls(url)

# Procesa y sube los datos de cada URL
process_and_upload_trip_data(filtered_urls)

Archivo Parquet subido a Cloud Storage: gs://upload-scraping-gcp/import/raw/yellow_tripdata_2024-01.parquet 
Archivo Parquet subido a Cloud Storage: gs://upload-scraping-gcp/import/raw/green_tripdata_2024-01.parquet 
Archivo Parquet subido a Cloud Storage: gs://upload-scraping-gcp/import/raw/fhvhv_tripdata_2024-01.parquet 
